In [165]:
import pandas as pd
import numpy as np
from functools import partial
import re

df = pd.read_excel('node-edges.xlsx', header=None, )
df.columns = df.iloc[61]
df = df.iloc[0:30]

count = 0
for i in range(0, len(df.columns)):
    if (i % 2 != 0):
        df.columns.values[i] = ("%02d" % count) + ': Destination'
        count += 1
        
slices = []
for i in range(2, len(df.columns), 2):
    s = df.iloc[:, i-2:i]
    s = s.dropna()
    s = s[s.iloc[:,1] != '-'] # edges are marked at entrance AND destination, so missing values can be removed
    slices.append(s)
    
slices_dict = {s.iloc[:,0].name : s for s in slices}

def extract_prefix(index, row):
    return " ".join(row[0].split(" ", index)[:index])

def extract_prefix_aqua_hideout(row):
    if 'Warps' in row[0] or 'B1F' in row[0]:
        return extract_prefix(4, row)
    else:
        return extract_prefix(3, row)
    
def extract_prefix_abandoned_ship(row):
    if 'B1F' in row[0] or 'R2' in row[0]:
        return extract_prefix(3, row)
    else:
        return extract_prefix(4, row)
    
def extract_prefix_meteor_falls(row):
    if 'B1F' in row[0] or 'Rooms' in row[0]:
        return extract_prefix(4, row)
    else:
        return extract_prefix(3, row)

def split_sub_nodes(node, extract_prefix):
    sub_nodes = []
    # current_group = []
    current_prefix = ""
    start = 0
    end = 0
    for row in node.iloc:
        name = extract_prefix(row)
        if current_prefix == name:
            end += 1
        else:
            sub_nodes.append(node.iloc[start:end])
            start = end
            end += 1
            current_prefix = name
    sub_nodes.append(node.iloc[start:end])
    # The first iteration adds an empty dataframe. This removes it.
    return sub_nodes[1:]


def process_sub_nodes(key_str, extract_prefix, slices_dict):
    sub_nodes = split_sub_nodes(slices_dict.pop(key_str), extract_prefix)
    key_num, key_name = key_str.split(':')
    
    for i, node in enumerate(sub_nodes):
        # This regex matches anything after
        new_key = re.sub('(?<=: ).*', extract_prefix(node.iloc[0]), key_str)
        node = node.rename({key_str: new_key}, axis='columns')
        slices_dict[new_key] = node

process_sub_nodes('00: Places of Interest', partial(extract_prefix, 2), slices_dict)
process_sub_nodes('12: Granite Cave', partial(extract_prefix, 3), slices_dict) 
process_sub_nodes('13: Abandoned Ship', extract_prefix_abandoned_ship, slices_dict)
process_sub_nodes('15: Meteor Falls', extract_prefix_meteor_falls, slices_dict)  
process_sub_nodes('18: Mt Pyre', partial(extract_prefix, 3), slices_dict) 
process_sub_nodes('20: Magma Hideout', partial(extract_prefix, 3), slices_dict)  
process_sub_nodes('21: Aqua Hideout', extract_prefix_aqua_hideout, slices_dict)
process_sub_nodes('24: Seafloor Cavern', partial(extract_prefix, 3), slices_dict) 
process_sub_nodes('25: Pokecenters', partial(extract_prefix, 2), slices_dict)
process_sub_nodes('26: Victory Road', partial(extract_prefix, 3), slices_dict) 
process_sub_nodes('28: E4 Pokecenter', partial(extract_prefix, 3), slices_dict) 
process_sub_nodes('31: Sky Pillar', partial(extract_prefix, 3), slices_dict) 

slices = slices_dict.values()

nodes = {}
edges = {}
for s in slices:
    node_name = s.iloc[:,0].name
    node_name = str.strip(node_name.split(':')[1]) # remove number and whitespace from name
    edges.update({e: node_name for e in s.iloc[:,0]})
    nodes.update({node_name: {'node_name': node_name,
                              'transitions': [{'entrance': e, 'destination': d} for e, d in s.iloc],
                              'visited':  False,
                              'previous': {}}})
    
class Graph:
    def __init__(self, nodes, edges):
        self.nodes = nodes
        self.edges = edges
        
graph = Graph(nodes, edges)

In [159]:
split_sub_nodes(slices_dict['02: Rustboro'], partial(extract_prefix, 2))[0].iloc[0]
'Aqua Hideout B1F NE'
'21: Aqua Hideout'
re.sub(':.*', ': Aqua Hideout B1F NE', '21: Aqua Hideout')
re.sub('(?<=: ).*', ': Aqua Hideout B1F NE', '21: Aqua Hideout')

'21: : Aqua Hideout B1F NE'

In [161]:
for s in slices:
    print(s.iloc[:,0].name)
# print([s.iloc[:,0].name for s in slices])

01: Oldale/Petalburg
02: Rustboro
03: Dewford
04: Slateport/Mauville/Verdanturf
05: Fallarbor/Lavaridge
06: Fortree/Lilycove
07: Mossdeep
08: Pacifidlog
09: Sootopolis
10: Petalburg Woods
11: Rusturf Tunnel
14: Jagged Pass
16: Mirage Tower
17: Route 119/123
19: Surf Hub
22: Lilycove Contest Hall
23: Lilycove Department Store
27: Waterfall Hub
29: Navel Rock
30: Dive Hub
00: Rustboro Gym
00: Mauville Gym
00: Lavaridge Gym
00: E4 Sidney
00: E4 Glacia
00: E4 Drake
00: Groudon
00: Ho-oh
00: Lugia
00: Regice
12: Granite Cave 1F
12: Granite Cave B1F
12: Granite Cave B2F
13: Abandoned Ship 1F Left
13: Abandoned Ship 1F Right
13: Abandoned Ship Rooms NE
13: Abandoned Ship Rooms SE
13: Abandoned Ship B1F
15: Meteor Falls 1F
15: Meteor Falls B1F Left
15: Meteor Falls B1F Right
18: Mt Pyre 2F
18: Mt Pyre 3F
18: Mt Pyre 4F
18: Mt Pyre 5F
20: Magma Hideout 1F
20: Magma Hideout 3F
21: Aqua Hideout B1F NE
21: Aqua Hideout B1F S
21: Aqua Hideout B2F
21: Aqua Hideout Warps 1
21: Aqua Hideout Warps 2
21

In [167]:
import copy

def graph_algo(src, dest, graph):
    if src == dest:
        return True
    src_node = graph.nodes[src]
    src_node['visited'] = True
    queue = [src_node]

    while len(queue) > 0:
        # print('Queue: ', sorted([x['node_name'] for x in queue]))
        current = queue.pop(0)
        # if current['node_name'] == dest:
            # print('Current: %s equals Dest: %s' % (current['node_name'], dest))
            # print('### FOUND IT ####')
            # print(current)
        for t in current['transitions']:
            node_name = graph.edges.get(t['destination'])
            # the node might not exist: Strength, one-way, the 32: Marts thing is not really present in the sheet.
            if node_name:
                node = graph.nodes[node_name]
                if node['visited']:
                    continue
                # add previous
                prev = node['previous']
                if prev.get(current['node_name']):
                    prev[current['node_name']].append(t)
                else:
                    prev[current['node_name']] = [t]
                # mark visited
                # if not node['visited']:
                    # node['visited'] = True
                    # queue.append(node)
                node['visited'] = True
                queue.append(node)
    return graph.nodes[dest]

graph_copy = copy.deepcopy(graph)
dest = graph_algo('Slateport/Mauville/Verdanturf', 'Rustboro', graph_copy)
dest

{'node_name': 'Rustboro',
 'transitions': [{'entrance': 'Rustboro Mart',
   'destination': 'Petalburg Woods SE'},
  {'entrance': 'Rustboro Pokecenter',
   'destination': 'Sootopolis Pokecenter S'},
  {'entrance': 'Rustboro SE', 'destination': 'Meteor Falls 1F E'},
  {'entrance': 'Rustboro Cut House', 'destination': 'Slateport Pokecenter S'}],
 'visited': True,
 'previous': {'Meteor Falls 1F': [{'entrance': 'Meteor Falls 1F E',
    'destination': 'Rustboro SE'}]}}

In [172]:
def flatten_prev(node_name, graph):
    node = graph.nodes[node_name]
    return [(node[0], trans) for node in list(node['previous'].items())
                                for trans in node[1]]
def list_prev(node_name, graph):
    node = graph.nodes[node_name]
    flat_prev = flatten_prev(node_name, graph)
    return flat_prev
    

print(flatten_prev('Rustboro', graph_copy)[0])
print(flatten_prev('Meteor Falls 1F', graph_copy)[0])
print(flatten_prev('Fallarbor/Lavaridge', graph_copy)[0])
# print(flatten_prev('Slateport/Mauville/Verdanturf', graph_copy)[0])
# print(flatten_prev('Slateport/Mauville/Verdanturf', graph_copy)[0])


list_prev('Slateport/Mauville/Verdanturf', graph_copy)

('Meteor Falls 1F', {'entrance': 'Meteor Falls 1F E', 'destination': 'Rustboro SE'})
('Fallarbor/Lavaridge', {'entrance': 'Fallarbor Fossil Maniac', 'destination': 'Meteor Falls 1F S'})
('Slateport/Mauville/Verdanturf', {'entrance': 'R109 Seashore House', 'destination': 'R114 Lanette'})


[]

In [189]:
print((graph_copy.nodes['Rustboro']['node_name'], graph_copy.nodes['Rustboro']['previous']))
print((graph_copy.nodes[list(graph_copy.nodes['Rustboro']['previous'].keys())[0]]['node_name'], graph_copy.nodes[list(graph_copy.nodes['Rustboro']['previous'].keys())[0]]['previous']))
# graph_copy.nodes['Rustboro']

('Rustboro', {'Meteor Falls 1F': [{'entrance': 'Meteor Falls 1F E', 'destination': 'Rustboro SE'}]})
('Meteor Falls 1F', {'Fallarbor/Lavaridge': [{'entrance': 'Fallarbor Fossil Maniac', 'destination': 'Meteor Falls 1F S'}]})


In [238]:
def list_path(dest, graph):
    node = graph.nodes[dest]
    path = []
    while node['previous']:
        path.append((node['node_name'], node['previous']))
        prev = list(node['previous'].keys())[0]
        node = graph.nodes[prev]
    path.append((node['node_name'], {}))
    return path

list_path('Rustboro', graph_copy)

[('Rustboro',
  {'Meteor Falls 1F': [{'entrance': 'Meteor Falls 1F E',
     'destination': 'Rustboro SE'}]}),
 ('Meteor Falls 1F',
  {'Fallarbor/Lavaridge': [{'entrance': 'Fallarbor Fossil Maniac',
     'destination': 'Meteor Falls 1F S'}]}),
 ('Fallarbor/Lavaridge',
  {'Slateport/Mauville/Verdanturf': [{'entrance': 'R109 Seashore House',
     'destination': 'R114 Lanette'}]}),
 ('Slateport/Mauville/Verdanturf', {})]

In [116]:
# TODO: obviously the edges are wrong
# renaming mesed up
# 07: Mossdeep
# 24-Seafloor Cavern R1: Seafloor Cavern
graph.edges['Granite Cave B1F SE']

KeyError: 'Granite Cave B1F SE'

In [ ]:
reachable = set()

def reach(node_name, graph, depth_max):
    node = graph.nodes[node_name]
    prev = node['previous'].keys()
    # print(prev)
    for k in prev:
        reachable.add(k)
        if depth_max > 0:
            reach(k, graph, depth_max-1)
reach('Rustboro', graph_copy, 5)
len(reachable)

In [ ]:
name = dest['node_name']
the_node = graph_copy.nodes[name]
prev = the_node['previous']
flat_prev = [[(node[0], trans)] for node in list(prev.items())
    for trans in node[1]]

def list_prev(node_name):
    node = graph_copy.nodes[node_name]
    flat_prev = [(node[0], trans) for node in list(node['previous'].items())
                                    for trans in node[1]]
    
    
flat_prev[0]